In [1]:
import requests
import datetime
import pandas as pd 
import numpy as np
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import random
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

### Crawl links from nhatot.com

In [2]:
# Set up Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Initialize the array to store the links
arr = []

for page in range(2):
    try:
        url = f"https://www.nhatot.com/mua-ban-nha-dat?page={page + 1}"
        driver.get(url)

        # Wait for JavaScript to render content
        driver.implicitly_wait(10)

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Find all the links with the specified class
        a_tags = soup.find_all("a", class_='AdItem_adItem__gDDQT')

        # Append the full link to the array
        for a in a_tags:
            full_link = "https://www.nhatot.com" + a['href']
            arr.append(full_link)

    except (NoSuchElementException, TimeoutException) as e:
        print(f"Error on page {page + 1}: {e}")

    # Introduce a random delay between requests
    time.sleep(random.uniform(5, 10))

# Close the WebDriver
driver.quit()

# Create a DataFrame from the array
df = pd.DataFrame(arr, columns=["Links"])

# Export the DataFrame to a CSV file
df.to_csv('nhatot.csv', index=True)

print("done")

## crawl data from each link

In [35]:
frame = pd.read_csv('nhatot.csv')
frame.head()

,Unnamed: 0,Links
0,0,https://www.nhatot.com/mua-ban-nha-dat-huyen-d...
1,1,https://www.nhatot.com/mua-ban-nha-dat-thanh-p...
2,2,https://www.nhatot.com/mua-ban-nha-dat-thanh-p...
3,3,https://www.nhatot.com/mua-ban-nha-dat-thanh-p...
4,4,https://www.nhatot.com/mua-ban-nha-dat-thanh-p...


In [36]:
address=[]
estate_condition,price_m2, main_direction,floor,id_house=[],[],[],[],[]
balcony_direction,Legal_papers,Character,acreage,bedroom,block=[],[],[],[],[],[]
bath_room,types,furniture_condition=[],[],[]
person,price=[],[]
items =['Tình trạng bất động sản:','Diện tích:','Giá/m2:','Số phòng ngủ:','Tên phân khu/Lô/Block/Tháp:','Tầng số:',
       'Số phòng vệ sinh:','Loại hình căn hộ:','Giấy tờ pháp lý:','Mã căn','Tình trạng nội thất:','Hướng cửa chính:',
       'Hướng ban công','Đặc điểm căn hộ']

In [37]:
def getdata(link):
    session = HTMLSession()
    url =link
    table_class = ".col-md-8"
    #r = requests.get(url)
    #if len(r.html.find(".notfound-message"))==0:
    r = session.get(url)
    if len(r.html.find(".notfound-message"))==0:
        arr=[]
        html_text = r.text
        r_html = HTML(html = html_text)
        r_table = r_html.find(table_class)
        if len(r_table) > 0:
            test = r_table[0]
            inf = test.find('.col-xs-12') 
            price = test.find('.priceWrapper___3ls0c')
            temp = r_html.find('a')
            col = temp[6].find('b') 
            info= inf[1].text+'\n' 
            arr.append(info) # dia chi nha
            arr.append(inf[3].text) # thong tin can nha
            arr.append(price[1].text) # gia nha
        else:
            return []
    
    str1 = arr
    str1=[arr]
    temp = arr[1].split('\n')
    idx=[]
    idx.append([arr[0].split('\n')[0]])
    for i in temp:
        idx.append(i.split('\n'))
        idx.append([arr[2]])
    return idx     

In [38]:
for link in frame['Links']:
    data = getdata(link)
    print(data)
    dem =0
    if (len(data)>0):
        address+=data[0]
        for item in items:
            mang=[]
            flag=0
            for i in data:
                if item in i[0]:
                    mang.append(i[0].split(':')[1])
                    flag = 1
            dem = dem+1
            if flag==1:
                if (dem ==1): estate_condition+=mang
                elif (dem ==2): acreage+=mang
                elif (dem ==3): price_m2+=mang
                elif (dem ==4): bedroom+=mang
                elif (dem ==5): block+=mang
                elif (dem ==6): floor+=mang
                elif (dem ==7): bath_room+=mang
                elif (dem ==8): types+=mang
                elif (dem ==9): Legal_papers+=mang
                elif (dem ==10): id_house+=mang
                elif (dem ==11): furniture_condition+=mang
                elif (dem ==12): main_direction+=mang
                elif (dem ==13): balcony_direction+=mang
                elif (dem ==14): Character+=mang
            else:
                if (dem ==1): estate_condition.append(np.nan)
                elif (dem ==2): acreage.append(np.nan)
                elif (dem ==3): price_m2.append(np.nan)
                elif (dem ==4): bedroom.append(np.nan)
                elif (dem ==5): block.append(np.nan)
                elif (dem ==6): floor.append(np.nan)
                elif (dem ==7): bath_room.append(np.nan)
                elif (dem ==8): types.append(np.nan)
                elif (dem ==9): Legal_papers.append(np.nan)
                elif (dem ==10): id_house.append(np.nan)
                elif (dem ==11): furniture_condition.append(np.nan)
                elif (dem ==12): main_direction.append(np.nan)
                elif (dem ==13): balcony_direction.append(np.nan)
                elif (dem ==14): Character.append(np.nan)  
        price+=data[len(data)-1]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [39]:
#print(address)
#print(estate_condition)
#print(acreage)
#print(price_m2)
#print(bedroom)
#print(block)
#print(floor)
#print(bath_room)
#print(types)
#print(Legal_papers)
#print(id_house)
#print(furniture_condition)
#print(main_direction)
#print(balcony_direction)
#print(Character)
#print(price)

In [40]:
df = {
        'DiaChi' : address,
        'TinhTrangBDS' : estate_condition,
        'DienTich' : acreage,
        'Gia/m2'   : price_m2,
        'Phongngu': bedroom,
      'TenPhanKhu': block,
      'SoTang': floor,
      'PhongTam':bath_room,
      'Loai': types,
      'GiayTo': Legal_papers,
      'MaCanHo': id_house,
      'TinhTrangNoiThat': furniture_condition,
      'HuongCuaChinh': main_direction,
      'HuongBanCong': balcony_direction,
      'DacDiem':Character,
      'Gia': price,
}
df = pd.DataFrame(df)
df.to_csv('raw_data.csv',encoding="utf-8-sig",index=False)
df

,DiaChi,TinhTrangBDS,DienTich,Gia/m2,Phongngu,TenPhanKhu,SoTang,PhongTam,Loai,GiayTo,MaCanHo,TinhTrangNoiThat,HuongCuaChinh,HuongBanCong,DacDiem,Gia


## Test

In [41]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_user_agent")
city ="Quận 1 , Tp Hồ Chí Minh"
country ="Việt Nam"
loc = geolocator.geocode(city+','+ country)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

ModuleNotFoundError: No module named 'geopy'

In [ ]:
arr=['a','b']
arr+=[]
arr